In [1]:
import asyncio

HOST = '127.0.0.1'
PORT = 1234

async def send_mcp_message(writer, package, content):
    message = f"#$#{package} {content}\n"
    writer.write(message.encode())
    await writer.drain()

async def mcp_client(oil_data):
    reader, writer = await asyncio.open_connection(HOST, PORT)

    # Negotiate MCP
    await send_mcp_message(writer, "mcp", "version: 2.1 to: 2.1")

    # Pretend to authenticate (if your server supports it)
    await send_mcp_message(writer, "mcp-auth", "type: client id: test_client")

    # Send oil data
    for entry in oil_data:
        code = entry["code"]
        price = entry["price"]
        ts = entry["created_at"]
        await send_mcp_message(writer, "oil-data", f":code={code} :price={price} :created_at={ts}")

    # Optionally close
    writer.close()
    await writer.wait_closed()
